In [2]:
import os
import pandas as pd

# def read_excel_files(folder_path):
#     files = os.listdir(folder_path)
#     dataframes = []
#     for file in files:
#         if file.endswith('.xlsx'):
#             file_path = os.path.join(folder_path, file)
#             df = pd.read_excel(file_path)
#             df['DATE'] = file.split('.')[0]
#             dataframes.append(df)
#     return pd.concat(dataframes, ignore_index=True)

# folders = ['07','08','09','10','11','12']
# dfs = []

# base_path = r'W:\CO\ДЕПАРТАМЕНТ ФИНАНСОВЫХ РИСКОВ И ПОРТФЕЛЬНОГО АНАЛИЗА\Управление финансовых рисков\Алтынай\ЦБ\2022'

# for folder in folders:
#     folder_path = os.path.join(base_path, folder)
#     if os.path.exists(folder_path):
#         df = read_excel_files(folder_path)
#         dfs.append(df)

# concatenated_df = pd.concat(dfs, ignore_index=True)



In [59]:
data=pd.read_csv(r"C:\Users\00059700\Desktop\NELLI_FXRATES.csv",delimiter=';')


In [60]:
data['USD']=data['USD'].str.replace(',','.').astype(float)
data['EUR']=data['EUR'].str.replace(',','.').astype(float)
data['CNY']=data['CNY'].str.replace(',','.').astype(float)
data['RUB']=data['RUB'].str.replace(',','.').astype(float)

In [61]:
data

,DATE,USD_QUANT,USD,EUR_QUANT,EUR,CNY_QUANT,CNY,RUB_QUANT,RUB
0,01.01.2019,1,384.20,1,439.37,1,55.88,1,5.52
1,02.01.2019,1,384.20,1,439.37,1,55.88,1,5.52
2,03.01.2019,1,384.20,1,439.37,1,55.88,1,5.52
3,04.01.2019,1,381.38,1,433.78,1,55.49,1,5.53
4,05.01.2019,1,378.12,1,431.13,1,55.08,1,5.53
...,...,...,...,...,...,...,...,...,...
2036,29.07.2024,1,474.83,1,515.38,1,65.48,1,5.50
2037,30.07.2024,1,473.29,1,512.81,1,65.21,1,5.47
2038,31.07.2024,1,475.37,1,514.97,1,65.56,1,5.51
2039,01.08.2024,1,473.95,1,513.38,1,65.64,1,5.50


In [62]:
data['DATE'] = pd.to_datetime(data['DATE'], format='%d.%m.%Y').dt.strftime('%Y-%m-%d')


### to concat excel sheets from excel file 

In [ ]:

# os.chdir(r'W:\CO\ДЕПАРТАМЕНТ ФИНАНСОВЫХ РИСКОВ И ПОРТФЕЛЬНОГО АНАЛИЗА\Управление финансовых рисков\Алтынай\ЦБ\2023')
# excel_files = [  'Портфель ЦБ 10 23.xlsx',
#  'Портфель ЦБ 11 23.xlsx',
#  'Портфель ЦБ 12 23.xlsx']

# dfs = []

# # Read all sheets from each file and append to the list
# for file in excel_files:
#     xl = pd.ExcelFile(file)
#     for sheet_name in xl.sheet_names:
#         df = xl.parse(sheet_name)
#         df['DATE'] = sheet_name  # Add a 'DATE' column with the sheet name
#         dfs.append(df)

# concatenated_df_new = pd.concat(dfs, ignore_index=True)

### to create table

In [64]:
def try_convert_to_datetime(series):
    try:
        return pd.to_datetime(series)
    except ValueError:
        return series

# Iterate over columns and try to convert them to different data types
for col in data.columns:
    data['DATE'] = try_convert_to_datetime(data['DATE'])

In [66]:
data.dtypes

DATE         datetime64[ns]
USD_QUANT             int64
USD                 float64
EUR_QUANT             int64
EUR                 float64
CNY_QUANT             int64
CNY                 float64
RUB_QUANT             int64
RUB                 float64
dtype: object

In [68]:
import pandas as pd
import cx_Oracle

# Connection details
dsn = cx_Oracle.makedsn('10.23.13.62', '1521', 'OPERRISKDB')
connection = cx_Oracle.connect('OPERRISK_DS', 'Qwerty2020##', dsn)
cursor = connection.cursor()

# Sample DataFrame

# Table name to be created
table_name = 'NELLI_FX_RATES_NEW'

# Dictionary mapping DataFrame dtypes to Oracle data types
data_types_mapping = {
    'int64': 'NUMBER',
    'float64': 'NUMBER',
    'object': 'VARCHAR2(255)',
    'datetime64[ns]': 'DATE'
}

# Generate the CREATE TABLE query
create_table_query = f"CREATE TABLE {table_name} ("

for column_name, dtype in data.dtypes.items():
    oracle_data_type = data_types_mapping.get(str(dtype), 'VARCHAR2(255)')
    
    # Check for valid column names
    if not column_name.strip():
        raise ValueError("Empty column name found in DataFrame")
    
    # Use double quotes for the DATE column name
    if column_name == 'DATE':
        column_name = 'DATE_'
    
    create_table_query += f"{column_name} {oracle_data_type}, "

create_table_query = create_table_query.rstrip(', ') + ")"

try:
    # Execute the CREATE TABLE query
    print(f"Executing query: {create_table_query}")
    cursor.execute(create_table_query)
    # Commit the changes
    connection.commit()
    print(f"Table '{table_name}' created successfully.")
except cx_Oracle.DatabaseError as e:
    error, = e.args
    print(f"Error {error.code}: {error.message}")
    print(f"Problematic query: {create_table_query}")

# Close the cursor and connection
cursor.close()
connection.close()

Executing query: CREATE TABLE NELLI_FX_RATES_NEW (DATE_ DATE, USD_QUANT NUMBER, USD NUMBER, EUR_QUANT NUMBER, EUR NUMBER, CNY_QUANT NUMBER, CNY NUMBER, RUB_QUANT NUMBER, RUB NUMBER)
Table 'NELLI_FX_RATES_NEW' created successfully.


In [70]:
data

,DATE,USD_QUANT,USD,EUR_QUANT,EUR,CNY_QUANT,CNY,RUB_QUANT,RUB
0,2019-01-01,1,384.20,1,439.37,1,55.88,1,5.52
1,2019-01-02,1,384.20,1,439.37,1,55.88,1,5.52
2,2019-01-03,1,384.20,1,439.37,1,55.88,1,5.52
3,2019-01-04,1,381.38,1,433.78,1,55.49,1,5.53
4,2019-01-05,1,378.12,1,431.13,1,55.08,1,5.53
...,...,...,...,...,...,...,...,...,...
2036,2024-07-29,1,474.83,1,515.38,1,65.48,1,5.50
2037,2024-07-30,1,473.29,1,512.81,1,65.21,1,5.47
2038,2024-07-31,1,475.37,1,514.97,1,65.56,1,5.51
2039,2024-08-01,1,473.95,1,513.38,1,65.64,1,5.50


### to insert data 

In [1]:
import airflow


C:\Users\00059700\AppData\Local\anaconda3\Lib\site-packages\airflow\__init__.py:36: RuntimeWarning: Airflow currently can be run on POSIX-compliant Operating Systems. For development, it is regularly tested on fairly modern Linux Distros and recent versions of macOS. On Windows you can run it via WSL2 (Windows Subsystem for Linux 2) or via Linux Containers. The work to add Windows support is tracked via https://github.com/apache/airflow/issues/10388, but it is not a high priority.
  warnings.warn(
OSError while attempting to symlink the latest log directory


In [3]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime

def print_hello():
    return "Hello, Airflow!"

with DAG('hello_airflow', description='Simple DAG', schedule_interval='@daily', start_date=datetime(2024, 1, 1), catchup=False) as dag:
    hello_task = PythonOperator(task_id='print_hello', python_callable=print_hello)

hello_task

C:\Users\00059700\AppData\Local\Temp\ipykernel_22548\504621435.py:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

C:\Users\00059700\AppData\Local\Temp\ipykernel_22548\504621435.py:8 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

<Task(PythonOperator): print_hello>

In [69]:
import pandas as pd
import cx_Oracle
import datetime

# Connection details
os.chdir(r"W:\CO\ДЕПАРТАМЕНТ ФИНАНСОВЫХ РИСКОВ И ПОРТФЕЛЬНОГО АНАЛИЗА\Управление финансовых рисков\Нелли")

dsn = cx_Oracle.makedsn('10.23.13.62', '1521', 'OPERRISKDB')
connection = cx_Oracle.connect('OPERRISK_DS', 'Qwerty2020##', dsn)
cursor = connection.cursor()

# Table name to insert data into
table_name = 'NELLI_FX_RATES_NEW'

try:
    for _, row in data.iterrows():
        placeholders = ', '.join([':' + str(i+1) for i in range(len(row))])
        values = []
        for val in row:
            if pd.isnull(val):
                values.append(None)
            elif isinstance(val, pd.Timestamp):  # Check if the value is a pandas Timestamp (datetime)
                values.append(val.to_pydatetime())
            else:
                values.append(str(val))  # Convert to string for non-datetime columns
        query = f"INSERT INTO {table_name} VALUES ({placeholders})"
        cursor.execute(query, values)

    # Commit the changes
    connection.commit()
    print("Data inserted successfully.")
except cx_Oracle.DatabaseError as e:
    error, = e.args
    print(f"Error {error.code}: {error.message}")
    print(f"Problematic query: {query}")

# Close the cursor and connection
cursor.close()
connection.close()

Data inserted successfully.


### to drop data

In [67]:
import pandas as pd
import cx_Oracle

dsn = cx_Oracle.makedsn('10.23.13.62', '1521', service_name='OPERRISKDB')
connection = cx_Oracle.connect('OPERRISK_DS', 'Qwerty2020##', dsn)

drop_query = "DROP TABLE NELLI_FX_RATES_NEW"

cursor = connection.cursor()
cursor.execute(drop_query)

DatabaseError: ORA-00942: table or view does not exist

### to store data from oracle table to df

In [1]:
import pandas as pd
import cx_Oracle

dsn = cx_Oracle.makedsn('10.23.13.62', '1521', service_name='OPERRISKDB')
connection = cx_Oracle.connect('OPERRISK_DS', 'Qwerty2020##', dsn)

select_query = "SELECT * FROM NELLI_CB_NEW_VERSION"

cursor = connection.cursor()
cursor.execute(select_query)

# Fetch the result of the query
data = cursor.fetchall()

# Get the column names from the cursor description
columns = [col[0] for col in cursor.description]

# Create a DataFrame from the fetched data and column names
df = pd.DataFrame(data, columns=columns)

# Close the cursor and connection
cursor.close()
connection.close()

# Now you have your data in a Pandas DataFrame 'df'

In [23]:
df['USD_NEW']=df['USD']*4

In [3]:
df['DATE_VALUE'].max()

Timestamp('2024-09-13 00:00:00')

In [ ]:
proxy_servers:
    http: http://proxy.halykbank.nb:8080
    https: http://proxy.halykbank.nb:8080


In [28]:
import cx_Oracle

# Establish a connection to the Oracle database
dsn = cx_Oracle.makedsn('10.23.13.62', '1521', service_name='OPERRISKDB')
connection = cx_Oracle.connect('OPERRISK_DS', 'Qwerty2020##', dsn)

# Create a cursor
cursor = connection.cursor()

# List of old and new column names
column_names = {

    '"DATE"': 'DATE_',
}

# Rename columns in the table
for old_name, new_name in column_names.items():
    try:
        cursor.execute(f"ALTER TABLE NELLI_FX_RATES_NEW RENAME COLUMN {old_name} TO {new_name}")
        connection.commit()
        print(f"Column {old_name} renamed to {new_name} successfully.")
    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print("Error: ", error)

# Close the cursor and connection
cursor.close()
connection.close()

Column "DATE" renamed to DATE_ successfully.


In [1]:
import pandas as pd
import cx_Oracle

dsn = cx_Oracle.makedsn('10.23.13.62', '1521', service_name='OPERRISKDB')
connection = cx_Oracle.connect('OPERRISK_DS', 'Qwerty2020##', dsn)

select_query = "SELECT table_name FROM user_tables"

cursor = connection.cursor()
cursor.execute(select_query)

# Fetch the result of the query
tables = [row[0] for row in cursor.fetchall()]


In [2]:
tables

['CREDIT_DATE',
 'TRANSER_CREDIT',
 'DEATH_TRANSFER',
 'EMP_200224',
 'CLIENT_SEND_EM',
 'HB_DOVEREN_NUMBER',
 'HB_DOVEREN_RESULT',
 'W4_PRODUCT_H',
 'W4_STATUS_H',
 'W4_TRANS_TYPE_H',
 'W4_PROD_EVENT_H',
 'W4_SIC_H',
 'BONUS_CLIENT_3X',
 'THIRD_SIGM',
 'TEMP_MAXLIMIT',
 'TMP_CANCEL',
 'NEW_COMMERCANT',
 'T_MAX_LIMIT',
 'LOG_ETL_PROCESS',
 'TMP_SOTRUDNIKI',
 'TMP_POKUPKI2',
 'EMPLOYEE_DBT',
 'CBS_PERSON_LEGAL_COPY',
 'MAIN_BONUS_XLS',
 'BUY_MYSELF',
 'OCRM_RB_SUBJECT_CONTACT_S',
 'T_MAXLIMIT',
 'TEST_TEMP',
 'T_CLIENTS_TMP',
 'TMP_TOP5',
 'SUM_9865',
 'ETL_EXCELL_2',
 'ETL_EXCELL_3',
 'TEST_SCRITP',
 'TEST_DEP_AGG',
 'HR_EMPLOYEE_NEW',
 'EMPLOYEE_PAYMENT',
 'EMPLOYEE_PAYMENT_2',
 'TMP_INC_FROD',
 'DS_DEPOSIT_DM',
 'DEPOSIT_FOR_CLIENT',
 'DEP_FOR_CL',
 'DEP_FOR_CL2',
 'W4_TRANSACTION_FL',
 'ZNOMOVE_TABLE',
 'ZNOMOVE_FIZ_ORDER',
 'JUR_ZNOMOVE',
 'JUR_ZNOMOVE_ORDER',
 'JUR_ZNOMOVE_2',
 'GM_SUBJECT_H',
 'OCRM_RB_SUBJECT_S',
 'GM_CURRENCY_H',
 'W4_CONTRACT_S',
 'W4_BIN_TABLE_H',
 'W4_MESS_C